<a href="https://colab.research.google.com/github/Gr-30/DL/blob/main/Assign4_Q2_Twitter_114_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Import Libraries/Dataset

## a. Import the required libraries and the dataset.

In [ ]:
import os
seed=21
os.environ['PYTHONHASHSEED']=str(seed)
import copy
import time
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
import tensorflow.keras.regularizers as reg
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/nnlm-en-dim128/2")
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import warnings
warnings.filterwarnings("ignore")

def reset_random_seeds(seed_val):
    os.environ['PYTHONHASHSEED']=str(seed_val)
    tf.random.set_seed(seed_val)
    np.random.seed(seed_val)

reset_random_seeds(seed)

## b. Check the GPU available

In [ ]:
!nvidia-smi -L

# Data Visualization

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Mtech DSE BITS/Sem3/DL/Assignment1/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", engine="python", header=None)
 # As the data has no column titles, we will add our own
data.columns = ["label", "time", "date", "query", "username", "text"]

## a. Print at least two movie reviews from each class of the dataset, for a sanity check that labels match the text.

In [ ]:
df_pos = data[data['label'] == 1]
df_neg = data[data['label'] == 0]

In [ ]:
print('POSITIVE Label data ')
df_pos.head()

In [ ]:
print('Negative Label data ')
df_neg.head()

#### Five top records of data

In [ ]:
data.head()

#### Five last records of data

In [ ]:
data.tail()

#### Coloumns/features in data

In [ ]:
data.columns

In [ ]:
data['label'].value_counts()

#### Length of data

In [ ]:
print('lenght of data is', len(data))

#### Shape of data

In [ ]:
data.shape

#### Data information

In [ ]:
data.info()

#### Data types of all coloumns

In [ ]:
data.dtypes

#### Checking Null values

In [ ]:
np.sum(data.isnull().any(axis=1))

#### Rows and columns in the dataset

In [ ]:
print('Count of columns in the data is:  ', len(data.columns))

In [ ]:
print('Count of rows in the data is:  ', len(data))

In [ ]:
data=data[['text','label']]

## b. Plot a bar graph of class distribution in a dataset. Each bar depicts the number of tweets belonging to a particular sentiment.

In [ ]:
datag = data.groupby('label').size().reset_index(name='count')
sns.barplot(y="count",x="label", data=datag)


## c. Other visualizations

In [ ]:
stopwords_ = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        collocations=False,
        background_color='white',
        stopwords=stopwords_,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
show_wordcloud(data['text'], "WordCloud for tweets")

# Data Pre-processing
a. Need for this Step-Since the models we use cannot accept string inputs or cannot be of the string format. We have to come up with a way of handling this step. The discussion of different ways of handling this step is out of the scope of this assignment.

b. Please use this pre-trained embedding layerfrom TensorFlow hub for this assignment. This link also has a code snippet on how to convert a sentence to a vector. Refer to that for further clarity on this subject.

c. Bring the train and test data in the required format.

## Assigning 1 to Positive sentment 4

In [ ]:
data['label'][data['label']==4]=1

##  Separating input feature and label

In [ ]:
X=data.text
y=data.label

### c. Bring the train and test data in the required format.

#### Separating the 80% data for training data and 20% for testing data
As we prepared all the tweets, now we are separating/splitting the tweets into training data and testing data.
- 80% tweets will be used in the training 
- 20% tweets will be used to test the performance of the model.

In [ ]:
X_train_org, X_test_org, Y_train, Y_test = train_test_split(data, y, test_size=0.2, random_state=2)
X_train_org, X_val_org, Y_train, Y_val = train_test_split(X_train_org, Y_train, test_size=0.2, random_state=3)

### b. Please use this pre-trained embedding layer from TensorFlow hub for this assignment. 

In [ ]:
X_train = embed(X_train_org.text.values).numpy()
X_val = embed(X_val_org.text.values).numpy()
X_test = embed(X_test_org.text.values).numpy()

# Data Visualization

## d. Print the shapes of train and test data.

In [ ]:
print('X_train: ', X_train.shape)
print('X_test: ', X_test.shape)
print('Y_train: ', Y_train.shape)
print('Y_test: ', Y_test.shape)

# Model Building
a. Sequential Model layers-Use AT LEAST 3 hidden layers with appropriate input for each. Choose the best number for hidden units and give reasons. : **To prepare near best model we gradually decrease number of neurons in hidden layers by dividing the number of neurons in the previous layers by 2. Hence when we start at 64 neurons and have to prepare 2 more layers then one option would be 32 followed by 16**

b. Add L2 regularization to all the layers.

c. Add one layer of dropout at the appropriate position and give reasons.d.Choose the appropriate activation function for all the layers.e.Print the model summary.

In [ ]:
def model_base(): #Defined tensorflow_based_model function for training tenforflow based model
    inputs = Input(name='inputs',shape=(128))
    layer = Dropout(0.1, name='dropout')(inputs)
    layer = Dense(64,name='layer1', activation='relu',kernel_regularizer=reg.L2(0.0005))(layer)
    layer = Dense(32,name='layer2', activation='relu', kernel_regularizer=reg.L2(0.0005))(layer) 
    layer = Dense(16,name='layer3', activation='relu', kernel_regularizer=reg.L2(0.0005))(layer) 
    layer = Dense(1,name='output', activation='sigmoid')(layer) 
    model = Model(inputs=inputs,outputs=layer) #here we are getting the final output value in the model for classification
    return model #function returning the value when we call it

callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=1,mode='auto', baseline=None, restore_best_weights=False
)

## e. Print the model summary

In [ ]:
model = model_base() # here we are calling the function of created model
print('Model summary')
model.summary() 

# Model Compilation
a.Compile the model with the appropriate loss function.

b.Use an appropriate optimizer. Give reasons for the choice of learning rate and its value.: **We intially choose 0.001(least possible) but later realised that the training is too slow and could be improved without compromising the accuracy of the model so changed the learning rate to 0.005.**

c.Use accuracy as a metric.

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.005),metrics=['accuracy']) 

# Model Training
a. Train the model for an appropriate number of epochs. Print the train and validation accuracy and loss for each epoch. Use the appropriate batch size.

b. Plot the loss and accuracy history graphs for both train and validation set. Print the total time taken for training.

In [ ]:
startTime = time.time()
history=model.fit(X_train,Y_train,batch_size=128,epochs=100, validation_data=(X_val, Y_val), callbacks=[callback])# here we are starting the training of model by feeding the training data
endTime = time.time()
print('Training finished in : {}s'.format(round(endTime - startTime, 2)))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'])

In [ ]:
print(history.history)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'])

# Model Evaluation

a.Print the final train and validation loss and accuracy. Print confusion matrix and classification report for the validation dataset. Analyse and report the best and worst performing class.

b.Print the two most incorrectly classified texts for each class in the test dataset.

In [ ]:
accr1 = model.evaluate(X_val,Y_val) #we are starting to test the model here

In [ ]:
print('Test set\n  Accuracy: {:0.2f}'.format(accr1[1])) #the accuracy of the model on test data is given below

In [ ]:
y_pred = model.predict(X_val) #getting predictions on the trained model
y_pred_class = (y_pred > 0.5)

In [ ]:
print('Classification Report')
print(classification_report(X_val, y_pred))
print("Confusion matrix")
CR=confusion_matrix(Y_val, y_pred_class)
print(CR)
fig, ax = plot_confusion_matrix(conf_mat=CR,figsize=(10, 10),
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()

### a.Negative class or Negative Sentiment is slightly better performing as per the confusion matrix.

#### ROC CURVE

In [ ]:
print(roc_auc_score(Y_val, y_pred_class))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_val, y_pred_class)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC CURVE')
plt.legend(loc="lower right")
plt.show()
print()

### b.Print the two most incorrectly classified texts for each class in the test dataset.

In [ ]:
def find_2_smallest_index(arr):
    first = 999
    second = 999
    first_index=0
    sexond_index=0
    for i in range(len(arr)):
        if arr[i] < first:
            second = copy.deepcopy(first)
            second_index = copy.deepcopy(first_index)
            first = copy.deepcopy(arr[i])
            first_index = copy.deepcopy(i)
        elif (arr[i] < second):
            second = copy.deepcopy(arr[i])
            second_index = copy.deepcopy(i)
    return first_index, second_index

In [ ]:
worst_2 = {}
y_pred_test = model.predict(X_test)
first_index, second_index = find_2_smallest_index(1-y_pred_test[:])
worst_2[0] = [X_test_org.text.values[first_index],X_test_org.text.values[second_index]]
first_index, second_index = find_2_smallest_index(y_pred_test[:])
worst_2[1] = [X_test_org.text.values[first_index],X_test_org.text.values[second_index]]
worst_2

# Hyperparameter Tuning
Build two more additional models by changing the following hyperparameters ONE at a time. Write the code for Model Building, Model Compilation, Model Training and Model Evaluation as given in the instructions above for each additional model.
1. Batch Size: Change the value of batch size in model training
2. Dropout: Change the position and value of dropout layer
Write a comparison between each model and give reasons for the difference in results.

### **1.Batch Size: Change the value of batch size in model training**

# Model Building
a. Sequential Model layers-Use AT LEAST 3 hidden layers with appropriate input for each. Choose the best number for hidden units and give reasons. : **To prepare near best model we gradually decrease number of neurons in hidden layers by dividing the number of neurons in the previous layers by 2. Hence when we start at 64 neurons and have to prepare 3 more layers then one option would be 32 followed by 16 followed by 8**

b. Add L2 regularization to all the layers.

c. Add one layer of dropout at the appropriate position and give reasons.d.Choose the appropriate activation function for all the layers.e.Print the model summary.

In [ ]:
def model_hyp1(): #Defined tensorflow_based_model function for training tensorflow based model
    inputs = Input(name='inputs',shape=(128))
    layer = Dropout(0.1, name='dropout')(inputs)
    layer = Dense(64,name='layer1', activation='relu',kernel_regularizer=reg.L2(0.0005))(layer)
    layer = Dense(32,name='layer2', activation='relu', kernel_regularizer=reg.L2(0.0005))(layer) 
    layer = Dense(16,name='layer3', activation='relu', kernel_regularizer=reg.L2(0.0005))(layer) 
    layer = Dense(8,name='layer4', activation='relu', kernel_regularizer=reg.L2(0.0005))(layer)
    layer = Dense(1,name='output', activation='sigmoid')(layer) 
    model = Model(inputs=inputs,outputs=layer) #here we are getting the final output value in the model for classification
    return model #function returning the value when we call it

callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=1,mode='auto', baseline=None, restore_best_weights=False
)

## e. Print the model summary

In [ ]:
model = model_hyp1() # here we are calling the function of created model
print('Model summary')
model.summary()

# Model Compilation
a.Compile the model with the appropriate loss function.

b.Use an appropriate optimizer. Give reasons for the choice of learning rate and its value.: **We intially choose 0.001(least possible) but later realised that the training is too slow and could be improved without compromising the accuracy of the model so changed the learning rate to 0.005.**

c.Use accuracy as a metric.

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.005),metrics=['accuracy']) 

# Model Training
a. Train the model for an appropriate number of epochs. Print the train and validation accuracy and loss for each epoch. Use the appropriate batch size.

b. Plot the loss and accuracy history graphs for both train and validation set. Print the total time taken for training.

In [ ]:
startTime = time.time()
history=model.fit(X_train,Y_train,batch_size=128,epochs=100, validation_split=0.2, callbacks=[callback])# here we are starting the training of model by feeding the training data
endTime = time.time()
print('Training finished in : {}s'.format(round(endTime - startTime, 2)))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'])

# Model Evaluation

a.Print the final train and validation loss and accuracy. Print confusion matrix and classification report for the validation dataset. Analyse and report the best and worst performing class.

b.Print the two most incorrectly classified texts for each class in the test dataset.

In [ ]:
accr1 = model.evaluate(X_val,Y_val) #we are starting to test the model here

In [ ]:
print('Test set\n  Accuracy: {:0.2f}'.format(accr1[1])) #the accuracy of the model on test data is given below

In [ ]:
y_pred = model.predict(X_val) #getting predictions on the trained model
y_pred_class = (y_pred > 0.5)

In [ ]:
print('Classification Report')
print(classification_report(X_val, y_pred))
print("Confusion matrix")
CR=confusion_matrix(Y_val, y_pred_class)
print(CR)
fig, ax = plot_confusion_matrix(conf_mat=CR,figsize=(10, 10),
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()

### a.Negative class or Negative Sentiment is slightly better performing as per the confusion matrix.

#### ROC CURVE

In [ ]:
print(roc_auc_score(Y_val, y_pred_class))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_val, y_pred_class)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC CURVE')
plt.legend(loc="lower right")
plt.show()
print()

### b.Print the two most incorrectly classified texts for each class in the test dataset.

In [ ]:
def find_2_smallest_index(arr):
    first = 999
    second = 999
    first_index=0
    sexond_index=0
    for i in range(len(arr)):
        if arr[i] < first:
            second = copy.deepcopy(first)
            second_index = copy.deepcopy(first_index)
            first = copy.deepcopy(arr[i])
            first_index = copy.deepcopy(i)
        elif (arr[i] < second):
            second = copy.deepcopy(arr[i])
            second_index = copy.deepcopy(i)
    return first_index, second_index

In [ ]:
worst_2 = {}
y_pred_test = model.predict(X_test)
first_index, second_index = find_2_smallest_index(1-y_pred_test[:])
worst_2[0] = [X_test_org.text.values[first_index],X_test_org.text.values[second_index]]
first_index, second_index = find_2_smallest_index(y_pred_test[:])
worst_2[1] = [X_test_org.text.values[first_index],X_test_org.text.values[second_index]]
worst_2

### **2.Optimiser: Use a different optimizer with the appropriate LR value**

# Model Building
a. Sequential Model layers-Use AT LEAST 3 hidden layers with appropriate input for each. Choose the best number for hidden units and give reasons. : **To prepare near best model we gradually decrease number of neurons in hidden layers by dividing the number of neurons in the previous layers by 2. Hence when we start at 64 neurons and have to prepare 2 more layers then one option would be 32 followed by 16**

b. Add L2 regularization to all the layers.

c. Add one layer of dropout at the appropriate position and give reasons.d.Choose the appropriate activation function for all the layers.e.Print the model summary.

In [ ]:
def model_hyp2(): #Defined tensorflow_based_model function for training tenforflow based model
    inputs = Input(name='inputs',shape=(128))
    layer = Dropout(0.1, name='dropout')(inputs)
    layer = Dense(64,name='layer1', activation='relu',kernel_regularizer=reg.L2(0.0005))(inputs)
    layer = Dense(32,name='layer2', activation='relu', kernel_regularizer=reg.L2(0.0005))(layer) 
    layer = Dense(16,name='layer3', activation='relu', kernel_regularizer=reg.L2(0.0005))(layer) 
    layer = Dense(1,name='output', activation='sigmoid')(layer) 
    model = Model(inputs=inputs,outputs=layer) #here we are getting the final output value in the model for classification
    return model #function returning the value when we call it

callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=1,mode='auto', baseline=None, restore_best_weights=False
)

## e. Print the model summary

In [ ]:
model = model_hyp1() # here we are calling the function of created model
print('Model summary')
model.summary()

# Model Compilation
a.Compile the model with the appropriate loss function.

b.Use an appropriate optimizer. Give reasons for the choice of learning rate and its value.: **We chose adam optimizer to incorporate the momentum component in the optimization process. We intially choose 0.001(least possible) but later realised that the training is too slow and could be improved without compromising the accuracy of the model so changed the learning rate to 0.005.**

c.Use accuracy as a metric.

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=0.005),metrics=['accuracy']) 

# Model Training
a. Train the model for an appropriate number of epochs. Print the train and validation accuracy and loss for each epoch. Use the appropriate batch size.

b. Plot the loss and accuracy history graphs for both train and validation set. Print the total time taken for training.

In [ ]:
startTime = time.time()
history=model.fit(X_train,Y_train,batch_size=128,epochs=100, validation_split=0.2, callbacks=[callback])# here we are starting the training of model by feeding the training data
endTime = time.time()
print('Training finished in : {}s'.format(round(endTime - startTime, 2)))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'])

In [ ]:
print(history.history)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'])

# Model Evaluation

a.Print the final train and validation loss and accuracy. Print confusion matrix and classification report for the validation dataset. Analyse and report the best and worst performing class.

b.Print the two most incorrectly classified texts for each class in the test dataset.

In [ ]:
accr1 = model.evaluate(X_val,Y_val) #we are starting to test the model here

In [ ]:
print('Test set\n  Accuracy: {:0.2f}'.format(accr1[1])) #the accuracy of the model on test data is given below

In [ ]:
y_pred = model.predict(X_val) #getting predictions on the trained model
y_pred_class = (y_pred > 0.5)

In [ ]:
print('Classification Report')
print(classification_report(X_val, y_pred))
print("Confusion matrix")
CR=confusion_matrix(Y_val, y_pred_class)
print(CR)
fig, ax = plot_confusion_matrix(conf_mat=CR,figsize=(10, 10),
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()

### a.Negative class or Negative Sentiment is slightly better performing as per the confusion matrix.

#### ROC CURVE

In [ ]:
print(roc_auc_score(Y_val, y_pred_class))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_val, y_pred_class)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC CURVE')
plt.legend(loc="lower right")
plt.show()
print()

### b.Print the two most incorrectly classified texts for each class in the test dataset.

In [ ]:
def find_2_smallest_index(arr):
    first = 999
    second = 999
    first_index=0
    sexond_index=0
    for i in range(len(arr)):
        if arr[i] < first:
            second = copy.deepcopy(first)
            second_index = copy.deepcopy(first_index)
            first = copy.deepcopy(arr[i])
            first_index = copy.deepcopy(i)
        elif (arr[i] < second):
            second = copy.deepcopy(arr[i])
            second_index = copy.deepcopy(i)
    return first_index, second_index

In [ ]:
worst_2 = {}
y_pred_test = model.predict(X_test)
first_index, second_index = find_2_smallest_index(1-y_pred_test[:])
worst_2[0] = [X_test_org.text.values[first_index],X_test_org.text.values[second_index]]
first_index, second_index = find_2_smallest_index(y_pred_test[:])
worst_2[1] = [X_test_org.text.values[first_index],X_test_org.text.values[second_index]]
worst_2

# Write a comparison between each model and give reasons for the difference in results.